[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/encoders/huggingface.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/encoders/huggingface.ipynb)

# Using HuggingFaceEncoder

HuggingFace is a huge ecosystem of open source models. It can be run locally and supports the largest library of encoders.

## Getting Started

We start by installing semantic-router with the `[local]` flag to include all necessary dependencies for `HuggingFaceEncoder`:

In [ ]:
!pip install -qU "semantic-router[local]==0.0.20"

We start by defining a dictionary mapping routes to example phrases that should trigger those routes.

In [1]:
from semantic_router import Route

politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president",
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

_**⚠️ If you see an ImportError, you must install local dependencies. You can do so by installing Semantic Router using `pip install -qU "semantic-router[local]"`.**_

Let's define another for good measure:

In [2]:
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

routes = [politics, chitchat]

Now we initialize our embedding model.

In [3]:
from semantic_router.encoders import HuggingFaceEncoder

encoder = HuggingFaceEncoder()

/Users/jamesbriggs/opt/anaconda3/envs/decision-layer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 1.06MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.05MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.43MB/s]
config.json: 100%|██████████| 612/612 [00:00<00:00, 2.90MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:01<00:00, 63.2MB/s]


Now we define the `RouteLayer`. When called, the route layer will consume text (a query) and output the category (`Route`) it belongs to — to initialize a `RouteLayer` we need our `encoder` model and a list of `routes`.

In [4]:
from semantic_router.layer import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes)

2024-01-09 00:22:35 INFO semantic_router.utils.logger Initializing RouteLayer


Now we can test it:

In [5]:
rl("don't you love politics?")

RouteChoice(name='politics', function_call=None)

In [6]:
rl("how's the weather today?")

RouteChoice(name='chitchat', function_call=None)

Both are classified accurately, what if we send a query that is unrelated to our existing `Route` objects?

In [7]:
rl("I'm interested in learning about llama 2")

RouteChoice(name=None, function_call=None)

In this case, we return `None` because no matches were identified.